This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ###


Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import requests


In [31]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

#Parameters
DATABASE = 'FSE' #Frankfurt Stock Exchange
DATASET = 'AFX_X' #Carl Zeiss Meditech
COLLAPSE = 'daily' 
LIMIT = 1 #Only one day

#Create url for API request
url = 'http://www.quandl.com/api/v3/datasets/{}/{}/?collapse={}&limit={}&apikey={}'.format(
    DATABASE,DATASET,COLLAPSE,LIMIT,API_KEY)
r = requests.get(url)

In [32]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())
print(r.json()['dataset']['column_names'])

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': 1, 'transform': None, 'column_index': None, 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': 'daily', 'order': None, 'database_id': 6129}}
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Unit

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [33]:
#Step 1: Collecting data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017.
#Create request for 2017 data
START_DATE = '2017-01-01'
END_DATE = '2017-12-31'
url = 'http://www.quandl.com/api/v3/datasets/{}/{}/?start_date={}&end_date={}&collapse={}&apikey={}'.format(
    DATABASE, DATASET, START_DATE, END_DATE, COLLAPSE, API_KEY)

#Get response object: r
r = requests.get(url)
 
#Step 2: Converting the returned JSON object into a Python dictionary.
json_data = r.json()

In [42]:
#Step 3: Calculating what the highest and lowest opening prices were for the stock in this period.

opening = [data[1] for data in json_data['dataset']['data']
        if data[1] != None]
print('Highest opening price = ${:.2f}'.format(max(opening)))
print('Lowest opening price = ${:.2f}'.format(min(opening)))

Highest opening price = $53.11
Lowest opening price = $34.00


In [44]:
#Step 4: Determining what was the largest change in any one day based on high and low price.

intraday = [data[2]-data[3] for data in json_data['dataset']['data']
        if data[1] != None]
print('Largest one day price increase based on high and low price = ${:.2f}'.format(max(intraday)))

Largest one day price increase based on high and low price = $2.81


In [48]:
#Step 5: Determining what was the largest change between two days based on closing price.

closing = [data[4] for data in json_data['dataset']['data']
        if data[4] != None]
delta_daily = [price - closing[i-1] for i, price in enumerate(closing)][1:]
print('Largest change between two days based on closing price = ${:.2f}'.format(max(delta_daily)))

Largest change between two days based on closing price = $2.56


In [51]:
#Step 6: Determining what was the average daily trading volume during this year.

traded_volume = [data[6] for data in json_data['dataset']['data']
        if data[6] != None]
print('Average daily trading volume = {:.0f}'.format(sum(traded_volume)/len(traded_volume)))

Average daily trading volume = 89124


In [53]:
#Step 7: Determining what was the median trading volume during this year.

#Create a function to determine median without 3rd party packages
def median(lst):
    n = len(lst)
    s = sorted(lst)
    return (sum(s[n//2-1:n//2+1])/2.0, s[n//2])[n % 2] if n else None

print('Median daily trading volume = {:.0f}'.format(median(traded_volume)))

Median daily trading volume = 76286
